Implemented on top of AzureML.
Reference: https://docs.microsoft.com/en-us/azure/machine-learning/

Pipelines are going to be created inside some Azure ML workspace.
If you already have one, you can get the reference:

In [ ]:
from azureml.core import Workspace

ws = Workspace.from_config()

In case if you do not have, any you can either create it on Azure portal or programmatically:

In [ ]:
from azureml.core import Workspace
ws = Workspace.create(name = # Name of your workspace
                      subscription_id = # Your subscription id 
                      resource_group = # resource group name
                      create_resource_group = False,
                      location = # location, i.e. 'eastus2' 
                     )
ws.get_details()

This is a library of pipeline steps / primitives for creating batch training pipelines from logs produced by personalization service. Also, you can find some recipe pipelines which can be either used as-is or as a reference.

For creating pipeline application context is required:

In [ ]:
import application
from application import context

ctx = context.Context(
    accountName = # Azure storage account name
    accountKey = # Azure storage key
    appId = # application name
    appFolder = # application folder
)

Now you can create batch training pipeline for vw batch training:

In [ ]:
import pipelines
from pipelines import vw_train

pipeline = vw_train.create_pipeline(ws, ctx)

Created pipeline can be either submitted for execution:

In [ ]:
from azureml.core import Experiment

pipeline_run = Experiment(ws, 'VW_train').submit(
    pipeline,
    pipeline_params = {
        "start_datetime":'02/13/2019 00:00:00',
        "end_datetime":'02/13/2019 23:59:59'})

print("Train Pipeline is submitted for execution")

Or published to be available via REST endpoint:

In [ ]:
published_pipeline = pipeline.publish(name="VW_train", description="Example vw train pipeline")
print(published_pipeline.id)
print(published_pipeline.endpoint)

Example of REST call:
Reference for better understanding of Azure ML authentication:
https://github.com/Azure/MachineLearningNotebooks/blob/f16bf27e26d6ff7a2cf76e8cbcda4ccc9c878a0c/how-to-use-azureml/manage-azureml-service/authentication-in-azureml/authentication-in-azure-ml.ipynb

In [ ]:

from azureml.core.authentication import InteractiveLoginAuthentication
import requests

auth = InteractiveLoginAuthentication()
aad_token = auth.get_authentication_header()

response = requests.post(published_pipeline.endpoint, 
    headers=aad_token, 
    json={"ExperimentName": "VW_train",
        "ParameterAssignments": {"start_datetime":'02/13/2019 00:00:00',"end_datetime":'02/13/2019 23:59:59'}})